In [ ]:
#%%capture
#%matplotlib inline
#!pip3 install pyriemann
#!pip3 install mne
#!conda create --name=new_environment_name python=3
#!conda activate new_environment_name

In [ ]:
import numpy as np
#--- 
import pandas as pd
import scipy.io as sio # Download mat files
from google.colab import drive # Mount the drive
from google.colab import files # Save the experimental results to the drive 
import sys # To print error messages
#---
#import mne # MNE and PyRiemann are tha main tools for classification
#from mne import io
#from mne.datasets import sample
#from pyriemann.estimation import XdawnCovariances
#from pyriemann.tangentspace import TangentSpace
#---
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import make_pipeline
#---
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
#from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.gaussian_process.kernels import Matern, RationalQuadratic, ExpSineSquared, DotProduct, WhiteKernel
#---
from sklearn.metrics import RocCurveDisplay # Plot the ROC 
from sklearn import metrics
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt2
from matplotlib.colors import ListedColormap
from sklearn.linear_model import RidgeCV
#--- 
from sklearn.preprocessing import MinMaxScaler # Import various scalers 
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
#--- 
from sklearn.decomposition import PCA         # PCA, transfoemations and grid search
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline # See above
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
# Three models selected and sorted. They return probabilistic values of classes
def classifiers_3():
  names = [
      "Logistic Regression",
      "Gaussian Process",
      "Naive Bayes",
      ]
  classifiers = [
      LogisticRegression(),
      GaussianProcessClassifier(1.0 * RBF(1.0)), 
      GaussianNB(),
      ]
  return names, classifiers
# --- 
# Basic kernels: RBF, Maternel, Rational Quadratic, Dot Product, Exponential Sine Quadratic.
# Composite kernels: White, Ridge, Periodic, Irregular, Noisy, Rumble.
def classifiers_GP():
  classifiers = [
      LogisticRegression(),
      GaussianProcessClassifier(1.0 * RBF(1.0)),
      GaussianProcessClassifier(kernel=Matern(length_scale=1.0, nu=1.5)),
      GaussianProcessClassifier(kernel=RationalQuadratic(length_scale=1.0, alpha=1.0, alpha_bounds=(1e-5, 1e5))),
      GaussianProcessClassifier(kernel=DotProduct(sigma_0=1.0, sigma_0_bounds=(1e-5, 1e5))),
      #GaussianProcessClassifier(kernel = 1.0 * ExpSineSquared(1.0, 5.0, periodicity_bounds=(1e-2, 1e1))),    
      #GaussianProcessClassifier(kernel=ExpSineSquared(length_scale=1.0, periodicity=1.0, periodicity_bounds=(1e-5, 1e5))), 
      # Composite
      #GaussianProcessClassifier(kernel = 1.0 * ExpSineSquared(1.0, 5.0, periodicity_bounds=(1e-2, 1e1)) + WhiteKernel(1e-1)),
      #GaussianProcessClassifier(kernel = ExpSineSquared()),
      #GaussianProcessClassifier(kernel = (2.0**2 * RBF(length_scale=100.0) * ExpSineSquared(length_scale=1.0, periodicity=1.0, periodicity_bounds="fixed"))),
      GaussianProcessClassifier(kernel = 0.5**2 * RationalQuadratic(length_scale=1.0, alpha=1.0)),
      GaussianProcessClassifier(kernel =  0.1**2 * RBF(length_scale=0.1) + WhiteKernel(noise_level=0.1**2, noise_level_bounds=(1e-5, 1e5))),
      #GaussianProcessClassifier(kernel =  44.8**2 * RBF(length_scale=51.6) + 2.64**2 * RBF(length_scale=91.5) * ExpSineSquared(length_scale=1.48, periodicity=1) + 0.536**2 * RationalQuadratic(alpha=2.89, length_scale=0.968) + 0.188**2 * RBF(length_scale=0.122) + WhiteKernel(noise_level=0.0367)),
      ]
  names = [
      "Logistic Regression",
      "Gaussian RBF",
      "GP Maternel",
      "GP Rational Quadratic",
      "GP Dot Product",
      #"Exponential Sine Quadratic",
      # Cpmposite
      #"GP White",
      #"GP Ridge",
      #"GP Periodic",
      "GP Irregular",
      "GP Noisy",
      #"GP Rumble",
      ]
  return names, classifiers
# --- Assert tne classifiers
names, classifiers = classifiers_3()

In [ ]:
# The data files are in the drive
drive.mount('/content/drive')
%cd  '/content/drive/MyDrive/EEG/'
%ls

Mounted at /content/drive
/content/drive/MyDrive/EEG
 archiveCodeApr14/
 archiveCodeApr28/
'archive EEG pdf'/
'Colab clone arl-eegmodels.git'
'Colab clone HTNet'
'Copy of EEG expert features S3 May 23 (1).ipynb'
'Copy of EEG expert features S3 May 23.ipynb'
 DataApr26/
 DataApr28/
'EEG class balance [Apr 19].gdoc'
'EEG classifier comparison Apr28.ipynb'
'EEG ERP experiment [Apr 12].gdoc'
'EEG ERP feature extraction [May 12].gdoc'
'EEG exhaustive model comparison [Apr 26].gdoc'
'EEG expert-engineered features [May 23].gdoc'
'EEG expert features May 16.ipynb'
'EEG expert features S3 May 23.ipynb'
 eeg_May19.mat
 eeg_May23_unfilt.mat
 EEG-model-comparison-basic.ipynb
'EEG PlotAppendix [May 2].pdf'
'EEG research plans [Apr 4].gdoc'
'EEG search over engineered features [May 30].gdoc'
'EEG SOW W1: Proof of Concept Analysis [May 5].gslides'
'EEG tables temporary.gsheet'
'EEG  three classifiers reported Apr30.ipynb'
'EEG time-window exploring [May 9].gdoc'
'EEG user behavior analysis [May 2].g

In [ ]:
# Expert-generated features
# Basic set 
def fea_basic():
  namel = ('PO9', 'PO10', 'PO7', 'PO8', 'PO7', 'PO8', 'AF7', 'AF7')
  elec  = (128, 42, 10, 39, 10, 39, 94, 94)
  elec  = tuple(_ - 1 for _ in elec) # Set of electrode indexes
  start = np.array([110, 110, 200, 200, 410, 410, 500, 400]) * 0.001
  stop  = np.array([160, 160, 410, 410, 650, 650, 800, 750]) * 0.001
  return namel, elec, start, stop
# Basic set with neighbours
def fea_nbr1():
  namel = ('PO9', 'PO10', 'PO7', 'PO7', 'PO7', 'PO8', 'PO8', 'PO8', 'PO7', 'PO7', 'PO7', 'PO8', 'PO8', 'PO8', 'AF7', 'AF7', 'AF7', 'AF7')
  elec  = (128, 42, 9, 10, 11, 38, 39, 40, 9, 10, 11, 38, 39, 40, 94, 95, 94, 95)
  elec  = tuple(_ - 1 for _ in elec) # Set of electrode indexes
  start = np.array([110, 110, 200, 200, 200, 200, 200, 200, 410, 410, 410, 410, 410, 410, 500, 500, 400, 400]) * 0.001
  stop  = np.array([160, 160, 410, 410, 410, 410, 410, 410, 650, 650, 650, 650, 650, 650, 800, 800, 750, 750]) * 0.001
  return namel, elec, start, stop
# Timing from the basic set for all electrodes
def fea_all():
  namel = ('ALL', 'ALL', 'ALL', 'ALL')#ALL
  elec  = ()
  start = np.array([110, 200, 410, 500]) * 0.001#, 400 
  stop  = np.array([160, 410, 650, 800]) * 0.001#, 750
  return namel, elec, start, stop
#FREQ_ = 256 # Hz for conversion
namel, elec, start, stop = fea_basic()

In [ ]:
# S3 dataset, the variables:
# 1) dataX, dataY, Users, timeY for each user assigned to each event
# 2) timeX, nameElec, posElec   for all users, the experiment setup 
mat_fname = 'eeg_May23_unfilt.mat'#'eeg_May19.mat'
mat_contents = sio.loadmat(mat_fname)
dataX = mat_contents['dataX']                   # The design matrix Events x Electrodes x Time
dataY = np.squeeze(mat_contents['dataY'])       # A class for each Event
dataU = np.squeeze(mat_contents['dataU'])       # A username for each Event
# timeY = mat_contents['timeY'] # Not needed here
# Predefined indexes of columns in the dataY structure, see the Matlab part 
# List of all available users in the data
users    = np.squeeze(mat_contents['nameUsers'])
timeline = np.squeeze(mat_contents['timeX'])
# posElec = mat_contents['posElec']   # Not needed here
# nameElec = mat_contents['nameElec'] # ditto
# Claffify each event (trial) as two classes Old word versus New word in the Recognition task

In [ ]:
# Updated after feat_aver for S3 data with embedded timeline (instead o freq)
def feat_mean(ts, start, stop, timeline):
  # Get a piece of time series ts
  # Example
  # ts = np.array([1, 2, 3, 11, 22, 33, 111, 222, 333, 1111, 2222, 3333])
  # timeline = range(0,len(ts))
  # start = -6 
  # stop = 35
  # len(ts) == len(timeline)
  # aver, feaseg = feat_mean(ts, start, stop, timeline)
  # print(feaseg)
  if start >= stop or start < timeline[0] or stop > timeline[-1]:
    sys.stderr.write(f'Start-stop mismatch the timeline! start: {start}, stop: {stop}, timeline[0]: {timeline[0]}, timeline[-1]: {timeline[-1]}')
  # Make a feature, here average it
  istart = np.searchsorted(timeline, start)
  istop  = np.searchsorted(timeline, stop, side='right') #- 1
  feaseg = ts[istart : istop]
  fea = np.average(feaseg)
  return fea
  # Also todo return the other features of a segment

In [ ]:
fX = []
for matX in dataX: # Process each event, the matrix electrodes, time
  for elc, sta, stp in zip(elec, start, stop):  # Get one time series
    fea = feat_mean(matX[elc,:], sta, stp, timeline)
    fX = np.append(fX,fea)
fX = np.reshape(fX, (len(dataX), len(elec)))

In [ ]:
# Prepare the design matrix and target vector 
# Four cases: 
# Correct events, All events
# Selected users, All users
# Expert electrodes, All electrodes
# ---
# Four classes 0 1 2 3  => two classes 0 1
# 0 Old correct
# 1 Old incorrect
# 2 New correct 
# 3 New incorrect
# --- 
# Select users
# userselect = users 
users_all = np.array([1037, 1363, 2038, 7977, 1045, 1368, 6639, 7980, 1034, 1327, 1385, 7974])
users_six = np.array([1158,1045,7980,1037,6639,1363]) # Six users with higher accuracy
#print(f'Selected: {np.isin(users, userselect)}')
class_all = np.array([0, 1, 2, 3])   # All four include incorrect responses
class_two = np.array([0, 2])         # Only correct responses


# --- Run the schedule 
users_list = (users_all, users_all, users_six, users_six)
class_list = (class_all, class_two, class_all, class_two)
for userselect, classelect in zip (users_list, class_list):
  # Select users and classes
  idx = (np.isin(dataU, userselect)) & (np.isin(dataY, classelect))
  # Make dataset
  Y = np.array([0 if _ <  2 else 1 for _ in dataY[idx]]) # Two classes in selected data
  X = fX[idx]
  print(f'Users: {userselect} --- Sample size: {len(Y)} --- Class balance: {sum(Y==0)} vs {sum(Y==1)}') # --- Events: {set(Y)} 
  classify(X, Y, names, classifiers)
  select_seatures(X,Y)


In [ ]:
# n_components = 5  # pick some components
def classify(X,Y, names, classifiers):
  for name, clf in zip(names, classifiers):
    print(f'Classifier: {name}')
    clf = make_pipeline(
        StandardScaler(),
        # PCA(),
        # XdawnCovariances(n_components),
        # TangentSpace(metric="riemann"),       
        clf, #LogisticRegression(),
        )
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    # --- 
    for train_idx, test_idx in cv.split(X): 
      preds = np.zeros(len(Y))
      y_train, y_test = Y[train_idx], Y[test_idx]
      clf.fit(X[train_idx], y_train)
      #preds[test_idx] = clf.predict(X[test_idx]) # Binary for accuracy
      preds[test_idx] = clf.predict_proba(X[test_idx])[::,1] # Probabilty for AUC
    fpr, tpr, _ = metrics.roc_curve(Y[test_idx], preds[test_idx])
    auc = round(metrics.roc_auc_score(Y[test_idx], preds[test_idx]), 4)
    plt.plot(fpr,tpr,label=name+", AUC="+str(auc))
  plt.rcParams['font.family'] = 'DejaVu Serif'
  plt.rcParams['lines.linewidth'] = 2
  #plt.rcParams['lines.markersize'] = 12
  plt.rcParams['xtick.labelsize'] = 12#24
  plt.rcParams['ytick.labelsize'] = 12#24
  plt.rcParams['legend.fontsize'] = 8#24
  #plt.rcParams['axes.titlesize'] = 36. 
  plt.rcParams['axes.labelsize'] = 12#24
  plt.gca().set_aspect('equal')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.legend()

In [ ]:
def select_seatures(X,Y):
  feature_names = ('PO9: 110-160', 'PO10: 110-160', 'PO7: 200-410', 'PO8: 200-410', 'PO7: 410-650', 'PO8: 410-650', 'AF7: 500-800', 'AF7: 400-750')
  feature_names = ('PO9-P1' ,'PO10-P1' ,'PO7-P2' ,'PO8-P2' ,'PO7-P3' ,'PO8-P3' ,'AF7-LNP' ,'AF7-LN')
  ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X, Y)
  importance = np.abs(ridge.coef_)
  feature_names = np.array(feature_names)
  plt.bar(height=importance, x=feature_names)
  plt.rcParams['font.family'] = 'DejaVu Serif'
  plt.ylabel('Feature importance')
  plt.rcParams['ytick.labelsize'] = 12#24
  plt.xlabel('Electrodes and peaks are features')
  plt.rcParams['xtick.labelsize'] = 9#24
  plt.show()